In [6]:
import os
import requests
from pathlib import Path
from tqdm import tqdm

# Configuration
MODEL_URL = "https://gpt4all.io/models/gguf/mistral-7b-openorca.Q4_0.gguf"
MODEL_NAME = "mistral-7b-openorca.Q4_0.gguf"
MODEL_DIR = Path("./models")
MODEL_PATH = MODEL_DIR / MODEL_NAME

def download_model_with_progress(url: str, dest: Path):
    """Download a model file with a progress bar."""
    if dest.exists():
        print(f"✅ Model already exists at {dest}")
        return

    headers = {"User-Agent": "Mozilla/5.0"}
    print(f"⬇️ Downloading model to: {dest}")

    with requests.get(url, headers=headers, stream=True) as r:
        r.raise_for_status()
        total_size = int(r.headers.get("Content-Length", 0))
        dest.parent.mkdir(parents=True, exist_ok=True)

        with open(dest, "wb") as f, tqdm(
            desc="📦 Downloading",
            total=total_size,
            unit="B",
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    bar.update(len(chunk))

    print("✅ Download complete.\n")

# Start the download
download_model_with_progress(MODEL_URL, MODEL_PATH)


⬇️ Downloading model to: models\mistral-7b-openorca.Q4_0.gguf


📦 Downloading: 100%|█████████████████████████████████████████████████████████████| 3.83G/3.83G [04:51<00:00, 14.1MB/s]

✅ Download complete.



In [2]:
from pathlib import Path
from gpt4all import GPT4All

MODEL_DIR = Path("./models")
MODEL_NAME = "mistral-7b-openorca.Q4_0.gguf"

def text_to_markdown(raw_text: str) -> str:
    gpt = GPT4All(model_name=MODEL_NAME, model_path=str(MODEL_DIR))

    prompt = f"""
You are a helpful assistant that converts plain text into rich and structured Markdown.
Use proper Markdown formatting for:
- Headings
- Task checkboxes
- Ordered and unordered lists
- Tables
- Images
- Blockquotes
- Inline and block math ($ and $$)
- Links
- Emphasis (*italic*, **bold**, ~~strikethrough~~)
- Footnotes

Input:
\"\"\"
{raw_text.strip()}
\"\"\"

Markdown output:
"""
    result = gpt.generate(prompt, max_tokens=1024)
    return result.strip()

if __name__ == "__main__":
    sample_text = """
    PROJECT ROADMAP
    Kickoff meeting scheduled
    
    Todo: allocate tasks
    Design UI
    Implement API
    DONE integrate CI/CD
    
    1) gather feedback
    2) iterate
    
    Image: mascot | https://example.com/img/mascot.png
    
    > “Code is like humor. When you have to explain it, it’s bad.”
    
    Here’s a link: https://github.com/spaCy/spaCy
    
    Some *italic*, some **bold**, some ~~obsolete~~.
    
    Inline math $E = mc^2$ and display:
    
    $$
    \\int_{a}^{b} f(x)dx
    $$
    
    | Stage | Owner | ETA |
    |-------|-------|-----|
    | Alpha | @alice | 2025-08-01 |
    
    Future Ideas
    Machine-learning recommendation engine
    
    [^1]: We’ll revisit after MVP.
    """

    markdown_output = text_to_markdown(sample_text)
    print(markdown_output)


# PROJECT ROADMAP

Kickoff meeting scheduled

Todo: allocate tasks
- Design UI
- Implement API
- DONE integrate CI/CD

1) gather feedback
2) iterate

Image: mascot | https://example.com/img/mascot.png

> “Code is like humor. When you have to explain it, it’s bad.”

Here's a link: [spaCy](https://github.com/spaCy/spaCy)

Some *italic*, some **bold**, some ~~obsolete~~.

Inline math $E = mc^2$ and display:

$$
\int_{a}^{b} f(x)dx
$$

| Stage | Owner   | ETA     |
|-------|---------|---------|
| Alpha | @alice  | 2025-08-01 |

Future Ideas
---------------------
Machine-learning recommendation engine

[^1]: We’ll revisit after MVP.


In [3]:
from llama_cpp import Llama
from pathlib import Path

MODEL_PATH = "./models/mistral-7b-instruct-v0.1.Q4_K_M.gguf"

# Initialize LLaMA model (load once)
llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=4096,
    n_threads=8,
    n_gpu_layers=35  # You can adjust based on your 3060 VRAM (~6-8 GB)
)

def text_to_markdown(text: str) -> str:
    prompt = f"""You are a Markdown generator.
Convert the following plain text to a rich Markdown document with:
- Proper headings
- Task lists
- Tables
- Links
- Images
- Quotes
- Footnotes
- Math formatting
- Code blocks
Use **bold** and *italic* where appropriate.

Input:
\"\"\"
{text.strip()}
\"\"\"

Markdown Output:
"""
    response = llm(prompt, max_tokens=1024, stop=["</s>"])
    return response["choices"][0]["text"].strip()

# --- Test it ---
if __name__ == "__main__":
    sample_text = """
PROJECT ROADMAP
Kickoff meeting scheduled

Todo: allocate tasks
Design UI
Implement API
DONE integrate CI/CD

Image: mascot | https://example.com/img/mascot.png

> “Code is like humor. When you have to explain it, it’s bad.”

Here's a link: https://github.com/spaCy/spaCy

Some *italic*, some **bold**, some ~~obsolete~~.

Inline math $E = mc^2$ and display:

$$
\\int_a^b f(x)dx
$$

| Stage | Owner | ETA |
|-------|-------|-----|
| Alpha | @alice | 2025-08-01 |

Future Ideas
Machine-learning recommendation engine

[^1]: We’ll revisit after MVP.
"""
    print(text_to_markdown(sample_text))


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./models/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 l

# PROJECT ROADMAP

Kickoff meeting scheduled

## Todo
- [ ] Allocate tasks
- [ ] Design UI
- [ ] Implement API
- [x] Integrate CI/CD

![Mascot](https://example.com/img/mascot.png)

> “Code is like humor. When you have to explain it, it’s bad.”

[Here's a link](https://github.com/spaCy/spaCy)

Some *italic*, some **bold**, some ~~obsolete~~.

Inline math:

$E = mc^2$

| Stage | Owner | ETA |
|-------|-------|-----|
| Alpha | @alice | 2025-08-01 |

## Future Ideas
- Machine-learning recommendation engine

[^1]: We’ll revisit after MVP.
